<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-итог" data-toc-modified-id="Общий-итог-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий итог</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Описание данных:**
- Признаки: `пол`, `возраст` и `зарплата` застрахованного, `количество членов семьи`.
- Целевой признак: `количество страховых выплат` клиенту за последние 5 лет

## Загрузка данных

Загрузим необходимые для работы библиотеки:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, r2_score

Загрузим данные и посмотрим общую информацию о датафрейме:

In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('insurance.csv')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Красота - пропусков нет. Глянем визуально:

In [3]:
data.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
3613,1,29.0,62600.0,0,0
1056,1,25.0,46700.0,3,0
2733,1,43.0,36800.0,0,1
1239,1,24.0,33300.0,2,0
138,1,25.0,44900.0,0,0


Настораживает столбец с полом (это единственный категориальный признак). А друг есть какой-либо третий пол. Проверим на всякий случай:

In [4]:
data['Пол'].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

Есть только девочки и мальчики, а данные подготовлены к обучению модели. Правда, мы не проверяли ещё на дубликаты. Посмотрим:

In [5]:
print('Количество явных дубликатов: ', data[data.duplicated() == True]['Пол'].count())

Количество явных дубликатов:  153


Около 3% дубликатов. Думаю можно удалить, но сперва взглянем на них поближе, чтобы убедиться в маловероятности совпадений:

In [6]:
display(data[data.duplicated(keep=False) == True].sort_values('Зарплата').head(10))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
4230,0,32.0,25600.0,1,0
1378,0,36.0,26400.0,0,0
2723,0,36.0,26400.0,0,0
1002,1,34.0,26900.0,0,0
1140,1,34.0,26900.0,0,0


Действительно. Главный признак - совпадение зарплат. Видно, что они не повторяются более двух раз. И при таком повторении дополнительно и пол, и возраст, и члены семьи совпадают. Крайне маловероятно, что это разные люди. Удалим дубликаты.

In [7]:
data = data.drop_duplicates().reset_index(drop=True)

print('Количество явных дубликатов: ', data[data.duplicated() == True]['Пол'].count())
print()
data.info()

Количество явных дубликатов:  0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


Всё прекрасно. Пропусков нет, повторов тоже. Не данные, а сказка.

### Вывод

Мы получили хороший пул данных, потребовалось лишь удалить явные дубликаты в размере 3% от объёма выборки. В остальном информация предоставлена нужных типов и подготовленная для обучения линейной регрессии.

## Умножение матриц

Ответим на вопрос:

**Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу?**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии **НЕ ИЗМЕНИТСЯ**.

**Обоснование:**

Рассмотрим с математической точки зрения, что произойдёт, если мы матрицу признаков умножим на обратимую (а значит, квадратную) матрицу. Введём необходимые обозначения:

- $w_i$ - вектор весов линейной регрессии после умножения признаков на матрицу $P$
- $a_i$ - предсказания после кодирования

Формула обучения примет вид:

$$
w_i = ((X P)^T X P)^{-1} (X P)^T y
$$

Выполним преобразования с использованием свойств обратных и транспонированных матриц:
- Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке.
- Обратное произведение квадратных матриц равно произведению обратных матриц, взятых в обратном порядке.
- Сочетательное свойство произведения матриц

Действие №1. $(X P)^T = P^T X^T$. Результат:
$$
w_i = (P^T X^T X P)^{-1} P^T X^T y
$$

Действие №2. Выделим скобками произведения матриц, которые входят в стандартную формулу обучения. Получаем:

$$
w_i = (P^T (X^T X) P)^{-1} P^T (X^T y)
$$

Действие №3. $(P^T (X^T X) P)^{-1} = (X^T X) P)^{-1} (P^T)^{-1}$. В итоге:

$$
w_i = (X^T X) P)^{-1} (P^T)^{-1} P^T (X^T y)
$$

Действие №4. $(X^T X) P)^{-1} = P^{-1} (X^T X)^{-1}$ и $(P^T)^{-1} P^T = E$. В результате:

$$
w_i = P^{-1} (X^T X)^{-1} E (X^T y)
$$

Действие №5. Выделим скобками произведения матриц, которые входят в стандартную формулу обучения. Получаем:

$$
w_i = P^{-1} (X^T X)^{-1} (E X^T) y
$$

Действие №6. $(E X^T) = X^T$ Формула ещё упрощается:

$$
w_i = P^{-1} (X^T X)^{-1} X^T y
$$

Действие №7. И о чудо! $(X^T X)^{-1} X^T y = w$. А значит веса после кодирования будут равны произведению обратной матрицы P и вектора весов без кодирования:

$$
w_i = P^{-1} w
$$

Какие же предсказания будут после кодирования и обучения? Проверим математически, применяя те же свойства:


Действие №1. Запишем формулу вектора предсказаний с учётом умножения матрицы признаков на обратимую матрицу:

$$
a_i = X P w_i
$$

Действие №2. Используем ранее полученную формулу $w_i = P^{-1} w$:

$$
a_i = X P P^{-1} w
$$

Действие №3. Видим $P P^{-1} = E$:

$$
a_i = X E w
$$

Действие №4. И $X E = X$:

$$
a_i = X w
$$

Действие №5. А $X w = a$:

$$
a_i = a
$$

### Вывод

Таким образом видим, что даже после умножения признаков на обратную матрицу, предсказания после обучения модели по кодированным признакам должно быть таким же. **Значит задача обучения не претерпит изменений, а веса линейной регрессии после преобразования будут результатом умножения обратной матрицы P и весов до преобразования**. Осталось проверить на практике.

## Алгоритм преобразования

**Алгоритм**

1. Создание случайной квадратной матрицы размерностью равной количеству признаков.
2. Проверка на обратимость матрицы
3. Если матрица необратима, повторяем пункты 1-2. Если обратима, переходим к следующему шагу.
4. Кодируем признаки умножением матрицы признаков на случайную обратимую матрицу и сохраняем результат в переменную в виде датафрейма.
5. Обучаем линейную регрессию на исходных и на зашифрованных данных
6. Сравниваем получившиеся метрики качества модели R2 на исходных и на зашифрованных данных.

**Обоснование**

1. При таком условии в результате получим матрицу кодированных признаков той же размерности, что и исходная.
2. Необходимая проверка, т.к. иначе математические преобразования описанные выше будут невозможны и получим ошибку обучения модели. Процесс проверки основан на применении np.linalg.inv() к сгенерированной матрице. Если она необратима, получим ошибку применения данной функции, а значит пригодится конструкция try - except. 

Остальное чистая техника.

При таком алгоритме кодирования, необходимо знать конкретную "кодирующую" матрицу, чтобы расшифровать данные. Так как мы получаем её случайно, подобрать её не представляется возможным.

При этом ключ к расшифровке нужно хранить за семью печатями, как и незашифрованные данные.

Запрограммируем алгоритм в преобразующую функцию:

In [9]:
def coding(features):
    # получаем матрицу из датафрейма
    features_matrix = features.values
    # получаем размерность кодирующей случайной матрицы
    size = features_matrix.shape[1]
    
    # проверяем обратимость матрицы вызовом np.linalg.inv. Если выдаст ошибку - матрица необратима,
    # тормозим функцию и выдаём соответствующее сообщение
    try:
        code_matrix = np.random.normal(size=(size, size))
        code_inv_matrix = np.linalg.inv(code_matrix)        
    except:
        # так как получить случайным образом необратимую матрицу крайне маловероятно, 
        # думаю будет достаточно подобной "заглушки" на такой случай
        print('Кодирующая матрица необратима, попробуйте ещё раз')
        return 
    
    # кодируем матрицу признаков
    coded_features_matrix = features_matrix @ code_matrix
    # преобразуем матрицу в датафрейм
    coded_features = pd.DataFrame(coded_features_matrix, columns=features.columns)
    
    return coded_features, code_matrix

## Проверка алгоритма

Проверим алгоритм кодирования с помощью кросс-валидации. Качество будет определено точнее и делить на валидационную и обучающую выборку не нужно - сэкономим строки кода. Для определения качества модели будем использовать метрику R2, создав скорер для функции кросс-валидации.

In [10]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

r2_scorer = make_scorer(r2_score)

model = LinearRegression()
cv_score_before = cross_val_score(model, features, target, cv=5, scoring=r2_scorer).mean()
print('Среднее качество модели до кодирования (кросс-валидация): ', cv_score_before)

Среднее качество модели до кодирования (кросс-валидация):  0.42779425802804927


Закодируем признаки и взглянем визуально на результат.

In [11]:
coded_features, code_matrix = coding(features)

display(coded_features.sample(5))
print()
print('Кодирующая матрица:')
print(code_matrix)

,Пол,Возраст,Зарплата,Члены семьи
1668,-22800.768466,-56804.916570,56724.860325,-58569.130567
2263,-19782.686828,-49186.357033,49092.934809,-50745.164118
4774,-24233.992060,-60376.965769,60291.539321,-62252.374830
2649,-16850.302676,-41919.932450,41846.615795,-43242.263847
4220,-20329.304615,-50613.346052,50533.929514,-52196.178165



Кодирующая матрица:
[[-0.66962432 -0.83519782  0.66281501  0.18700574]
 [-1.74873282 -0.11275565 -0.88928051 -1.48560742]
 [-0.47696298 -1.19079506  1.18970854 -1.22699624]
 [ 0.4931994  -0.27371474 -0.00524881  0.90267234]]


Кодирование прошло успешно. Столбец "Пол" об этом особенно громко заявляет. Возможных реальных значений всего два, а у нас в пяти случайных строках разные значения.

Проверим качество модели на закодированных признаках и сравним с качеством на незакодированных.

In [12]:
cv_score_after = cross_val_score(model, coded_features, target, cv=5, scoring=r2_scorer).mean()

print('Среднее качество модели до кодирования (кросс-валидация): ', cv_score_before)
print('Среднее качество модели после кодирования (кросс-валидация): ', cv_score_after)
print('Разница в качестве до и после кодирования: ', abs(cv_score_after - cv_score_before))

Среднее качество модели до кодирования (кросс-валидация):  0.42779425802804927
Среднее качество модели после кодирования (кросс-валидация):  0.4277942580281099
Разница в качестве до и после кодирования:  6.061817714453355e-14


Плохие новости - качество разное. Хорошие новости - разница ооочень маленькая. Думаю, что она получается из-за неодинакового разделения на выборки при работе функции кросс-валидации до кодирования и после. А может и округление где-то сыграло роль.

## Общий итог

Мы получили хороший пул данных, потребовалось лишь удалить явные дубликаты в размере 3% от объёма выборки. В остальном информация предоставлена нужных типов и подготовленная для обучения линейной регрессии.

Далее доказали, что даже после умножения признаков на обратную матрицу, предсказания после обучения модели по кодированным признакам должны быть такими же. **А значит задача обучения не претерпит изменений**.

**При этом веса линейной регрессии после преобразования будут результатом умножения обратной матрицы P и весов до преобразования**

$$
w_i = P^{-1} w
$$

Далее написали алгоритм кодирования данных и проверили качество моделей до кодирования признаков и после. Результат проверки такой:

In [13]:
print('Среднее качество модели до кодирования (кросс-валидация): ', cv_score_before)
print('Среднее качество модели после кодирования (кросс-валидация): ', cv_score_after)
print('Разница в качестве до и после кодирования: ', abs(cv_score_after - cv_score_before))

Среднее качество модели до кодирования (кросс-валидация):  0.42779425802804927
Среднее качество модели после кодирования (кросс-валидация):  0.4277942580281099
Разница в качестве до и после кодирования:  6.061817714453355e-14


Получившаяся разница метрик R2 до и после кодирования очень мала, а значит можно утверждать, что **кодирование прошло успешно и не навредило качеству модели.**